In [2]:
import nltk

In [19]:
corpus=nltk.corpus.nps_chat

In [32]:
f=corpus.open()

IOError: No such file or directory: u'/Users/nadapzy/nltk_data/corpora/nps_chat/str'

In [24]:
sentq=[]

In [25]:
for sent in corpus.posts():
    if sent[-1]=='?':
        sentq.append(1)
    else: sentq.append(0)

In [7]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
def preprocessor(desc):
    return re.sub("[^a-zA-Z]"," ", desc.lower())

In [9]:
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,lowercase=True,    \
                             preprocessor = None, \
                             stop_words =None,\
                             max_features = None,\
                             ngram_range=(1,1),min_df=2)   
                             #potential set binary=True
                            #potentially set preprocessor to remove all numbers                              
                            #stop words list:  list(stopwords.words("english"))  

In [13]:
def corpus_words(corpus):
    corpus_bows = vectorizer.fit_transform(corpus.sents())
    #hcpcs[hcpcs.isin()]
    #cardio_bows= vectorizer.transform()
    print('Print the list of all words in the bag...')
    vocab=vectorizer.get_feature_names()
    print(vocab)

In [14]:
corpus_words(corpus)

AttributeError: 'list' object has no attribute 'lower'

In [18]:
f=corpus.open('str')

IOError: No such file or directory: u'/Users/nadapzy/nltk_data/corpora/movie_reviews/str'